In [ ]:
from pathlib import Path
from collections import defaultdict
import os, re, subprocess

# ✏️ 경로 설정
CUR_DIR = Path(r"D:\golfDataset\dataset\test")  # 프로젝트 루트, dataset 폴더
FPS = 30  # 출력 비디오 FPS

def images_to_video(img_dir: Path, video_dir: Path, fps: int = FPS) -> None:
    """
    <prefix>_0000.jpg 묶음을 MP4로 변환하고,
    변환된 JPG 파일은 모두 삭제한다.
    """
    video_dir.mkdir(parents=True, exist_ok=True) # 비디오 저장 폴더 생성

    jpgs = [f for f in os.listdir(img_dir) if f.lower().endswith(".jpg")]
    pat = re.compile(r"(.+)_\d{4}\.jpg")  # ex) swingA_0003.jpg
    groups = defaultdict(list)
    for f in jpgs:
        m = pat.match(f)
        if m:
            groups[m.group(1)].append(f)

    for prefix, files in groups.items():
        files.sort()
        list_txt = img_dir / "list.txt"
        with open(list_txt, "w", encoding="utf-8") as fp:
            for f in files:
                fp.write(f"file '{img_dir/f}'\n")

        out_mp4 = video_dir / f"{prefix}.mp4"
        cmd = [
            "ffmpeg",
            "-y",
            "-f", "concat", "-safe", "0",
            "-i", str(list_txt),
            "-r", str(fps),
            "-pix_fmt", "yuv420p",
            str(out_mp4)
        ]

        try:
            subprocess.run(cmd, check=True)
            print(f"▶ {out_mp4.name}  ({len(files)} frames)")

            # --- 변환 성공 시 JPG 삭제 ---
            for f in files:
                (img_dir / f).unlink()
            print(f"🗑️  Deleted {len(files)} JPGs for '{prefix}'")

        except subprocess.CalledProcessError as e:
            print(f"❌ ffmpeg failed for '{prefix}': {e}")

        finally:
            list_txt.unlink(missing_ok=True)

# ✏️ true와 false 폴더 모두 처리
folders_to_process = ["true", "false"]

for folder_name in folders_to_process:
    ROOT_DIR = CUR_DIR / folder_name
    IMG_DIR = ROOT_DIR / "jpg"
    VIDEO_DIR = ROOT_DIR / "video"

    print(f"\n--- Processing '{folder_name}' folder ---")
    if IMG_DIR.exists():
        images_to_video(IMG_DIR, VIDEO_DIR)
    else:
        print(f"'{IMG_DIR}' does not exist. Skipping.")

print("\n모든 분류 폴더의 비디오 변환 및 JPG 삭제 완료.")


--- Processing 'true' folder ---
'D:\golfDataset\dataset\true\jpg' does not exist. Skipping.

--- Processing 'false' folder ---
'D:\golfDataset\dataset\false\jpg' does not exist. Skipping.

모든 분류 폴더의 비디오 변환 및 JPG 삭제 완료.
